# Import

In [1]:
import os
import random

import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

from sklearn.metrics import log_loss
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.metrics import log_loss
import torch.nn as nn
import timm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import torch

# Hyperparameter Setting

In [3]:
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 64,
    'EPOCHS': 50,
    'LEARNING_RATE': 1e-4,
    'SEED' : 42
}

# Fixed RandomSeed

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

# CustomDataset

In [5]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if is_test:
            # 테스트셋: 라벨 없이 이미지 경로만 저장
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith(('.jpg')):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            # 학습셋: 클래스별 폴더 구조에서 라벨 추출
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith(('.jpg')):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            # image = Image.open(img_path).convert('RGB')
            image = Image.open(img_path).convert("L") # 흑백으로 이미지 읽기
            image = np.array(image)  # PIL → ndarray로 변환
            if self.transform:
                image = self.transform(image=image)['image']  # dict 반환 → 'image' 키로 꺼냄
            return image
        else:
            img_path, label = self.samples[idx]
            # image = Image.open(img_path).convert('RGB')
            image = Image.open(img_path).convert("L") # 흑백으로 이미지 읽기
            image = np.array(image)  # PIL → ndarray로 변환
            if self.transform:
                image = self.transform(image=image)['image']  # dict 반환 → 'image' 키로 꺼냄
            return image, label


# Data Load

In [6]:
train_root = './data/train'
test_root = './data/test'

In [7]:
def repeat_1channel_to_3(image, **kwargs):
    return np.repeat(image[:, :, np.newaxis], 3, axis=2)
# 학습 데이터에 적용할 흑백 데이터 증강 파이프라인

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    
    # 1. 흑백 변환
    A.ToGray(p=1.0),
    
    # 2. 1채널 이미지를 3채널로 복제 (모델 입력 호환)
    A.Lambda(image=repeat_1channel_to_3),

    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomShadow(p=0.2),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.2),
    A.CoarseDropout(max_holes=2, max_height=16, max_width=16, p=0.3),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

def repeat_1channel_to_3(image, **kwargs):
    return np.repeat(image[:, :, np.newaxis], 3, axis=2)
# 학습 데이터에 적용할 흑백 데이터 증강 파이프라인

train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    
    # 1. 흑백 변환
    A.ToGray(p=1.0),
    
    # 2. 1채널 이미지를 3채널로 복제 (모델 입력 호환)
    A.Lambda(image=repeat_1channel_to_3),

    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.HueSaturationValue(p=0.2),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomShadow(p=0.2),
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.2),
    A.CoarseDropout(max_holes=2, max_height=16, max_width=16, p=0.3),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.ToGray(p=1.0),
    A.Lambda(image=lambda x: np.repeat(x[:, :, np.newaxis], 3, axis=2)),
    A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ToTensorV2()
])



c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\albumentations\core\validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\user\AppData\Local\Temp\ipykernel_15764\1111706092.py:20: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=2, max_height=16, max_width=16, p=0.3),
C:\Users\user\AppData\Local\Temp\ipykernel_15764\1111706092.py:44: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=2, max_height=16, max_width=16, p=0.3),
C:\Users\user\AppData\Local\Temp\ipykernel_15764\1111706092.py:52: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  A.Lambda(image=lambda x: np.repeat(x[:, :, np.newaxis], 3, axis=2)),


In [8]:
# 전체 데이터셋 로드
full_dataset = CustomImageDataset(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


In [9]:
## 흑백 전환 여부 확인용 추가 코드 ##
"""
import matplotlib.pyplot as plt

# train_loader에서 하나의 배치를 가져오기 ->  흑백으로 이미지 제대로 바꿨는지 확인하는 용도도
data_iter = iter(train_loader)
images_train, labels = next(data_iter) # 첫 번째 배치의 이미지와 레이블을 가져오기

# 첫 번째 이미지의 차원(채널 수) 확인
print(f"Shape of first image: {images_train[0].shape}")

# 첫 번째 이미지 시각화 (흑백일 경우 cmap='gray'로 출력)
plt.imshow(images_train[0].squeeze(), cmap='gray')  # .squeeze()로 (1, H, W) -> (H, W)로 변환
plt.title(f"Label: {labels[0]}")
plt.axis('off') 
plt.show()

# val_loader에서 하나의 배치를 가져오기 ->  흑백으로 이미지 제대로 바꿨는지 확인하는 용도도
data_iter_val = iter(val_loader)
images_val, labels = next(data_iter_val) # 첫 번째 배치의 이미지와 레이블을 가져오기

# 첫 번째 이미지의 차원(채널 수) 확인
print(f"Shape of first image: {images_val[0].shape}")

# 첫 번째 이미지 시각화 (흑백일 경우 cmap='gray'로 출력)
plt.imshow(images_val[0].squeeze(), cmap='gray')  # .squeeze()로 (1, H, W) -> (H, W)로 변환
plt.title(f"Label: {labels[0]}")
plt.axis('off') 
plt.show()
"""


'\nimport matplotlib.pyplot as plt\n\n# train_loader에서 하나의 배치를 가져오기 ->  흑백으로 이미지 제대로 바꿨는지 확인하는 용도도\ndata_iter = iter(train_loader)\nimages_train, labels = next(data_iter) # 첫 번째 배치의 이미지와 레이블을 가져오기\n\n# 첫 번째 이미지의 차원(채널 수) 확인\nprint(f"Shape of first image: {images_train[0].shape}")\n\n# 첫 번째 이미지 시각화 (흑백일 경우 cmap=\'gray\'로 출력)\nplt.imshow(images_train[0].squeeze(), cmap=\'gray\')  # .squeeze()로 (1, H, W) -> (H, W)로 변환\nplt.title(f"Label: {labels[0]}")\nplt.axis(\'off\') \nplt.show()\n\n# val_loader에서 하나의 배치를 가져오기 ->  흑백으로 이미지 제대로 바꿨는지 확인하는 용도도\ndata_iter_val = iter(val_loader)\nimages_val, labels = next(data_iter_val) # 첫 번째 배치의 이미지와 레이블을 가져오기\n\n# 첫 번째 이미지의 차원(채널 수) 확인\nprint(f"Shape of first image: {images_val[0].shape}")\n\n# 첫 번째 이미지 시각화 (흑백일 경우 cmap=\'gray\'로 출력)\nplt.imshow(images_val[0].squeeze(), cmap=\'gray\')  # .squeeze()로 (1, H, W) -> (H, W)로 변환\nplt.title(f"Label: {labels[0]}")\nplt.axis(\'off\') \nplt.show()\n'

# Model Define

In [10]:
""" # 기본 3채널 컬러 이미지일때
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        self.backbone = models.resnet18(pretrained=True)  # ResNet18 모델 불러오기
        self.feature_dim = self.backbone.fc.in_features 
        self.backbone.fc = nn.Identity()  # feature extractor로만 사용
        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기

    def forward(self, x):
        x = self.backbone(x)       
        x = self.head(x) 
        return x
"""

' # 기본 3채널 컬러 이미지일때\nclass BaseModel(nn.Module):\n    def __init__(self, num_classes):\n        super(BaseModel, self).__init__()\n        self.backbone = models.resnet18(pretrained=True)  # ResNet18 모델 불러오기\n        self.feature_dim = self.backbone.fc.in_features \n        self.backbone.fc = nn.Identity()  # feature extractor로만 사용\n        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기\n\n    def forward(self, x):\n        x = self.backbone(x)       \n        x = self.head(x) \n        return x\n'

In [11]:
class BaseModel(nn.Module):
    def __init__(self, num_classes):
        super(BaseModel, self).__init__()
        # ResNet18 모델 불러오기
        self.backbone = models.resnet18(pretrained=True)

        self.feature_dim = self.backbone.fc.in_features
        self.backbone.fc = nn.Identity()  # feature extractor로만 사용
        self.head = nn.Linear(self.feature_dim, num_classes)  # 분류기

    def forward(self, x):
        x = self.backbone(x)  # Backbone에서 feature 추출
        x = self.head(x)  # 분류기
        return x


# Train/ Validation

In [12]:
model = BaseModel(num_classes=len(class_names)).to(device)
best_logloss = float('inf')

# 손실 함수
criterion = nn.CrossEntropyLoss()

# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])

# 학습 및 검증 루프
for epoch in range(CFG['EPOCHS']):
    # Train
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)  # logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # Accuracy
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))

    # 결과 출력
    print(f"Train Loss : {avg_train_loss:.4f} || Valid Loss : {avg_val_loss:.4f} | Valid Accuracy : {val_accuracy:.4f}%")

    # Best model 저장
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        torch.save(model.state_dict(), f'best_model.pth')
        print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.4f})")

c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
[Epoch 1/10] Training:   0%|          | 0/415 [00:00<?, ?it/s]c:\Users\user\miniconda3\envs\HAI\Lib\site-packages\albumentations\core\transforms_interface.py:310: UserWarning: The image is already gray.
  target_function(ensure_contiguous_output(arg), **params),
[Epoch 1/10] Validation:   0%|          | 0/104 [00:00<?, ?it/s]


TypeError: <lambda>() got an unexpected keyword argument 'shape'

# Inference

In [95]:
test_dataset = CustomImageDataset(test_root, transform=val_transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

In [96]:
# 저장된 모델 로드
model = BaseModel(num_classes=len(class_names))
model.load_state_dict(torch.load('best_model.pth', map_location=device))
model.to(device)

# 추론
model.eval()
results = []

with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)

        # 각 배치의 확률을 리스트로 변환
        for prob in probs.cpu():  # prob: (num_classes,)
            result = {
                class_names[i]: prob[i].item()
                for i in range(len(class_names))
            }
            results.append(result)
            
pred = pd.DataFrame(results)

c:\Users\user\miniconda3\envs\project310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\miniconda3\envs\project310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


# Submission

In [97]:
submission = pd.read_csv('./sample_submission.csv', encoding='utf-8-sig')

# 'ID' 컬럼을 제외한 클래스 컬럼 정렬
class_columns = submission.columns[1:]
pred = pred[class_columns]

submission[class_columns] = pred.values
submission.to_csv('baseline_submission_v3.csv', index=False, encoding='utf-8-sig')